In [1]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics, preprocessing
import numpy as np

In [2]:
dataset = pd.read_csv ('test_iot1.csv')
df = dataset.drop(columns =['Epoch Time', 'Time', 'Source', 'Destination'])
le = preprocessing.LabelEncoder()
strings_only = df.select_dtypes(include=[object])
len(df)

1921098

In [3]:
# df_transformed = strings_only.apply(le.fit_transform)
# enc = preprocessing.OneHotEncoder()
# enc.fit(df_transformed)
for col in strings_only:
    strings_only[col+' Num'] = [hash(val) for val in strings_only[col]]
df_2 = df.drop(columns =['MAC Destination', 'Protocol', 'Info'])
# df_3 = df_transformed.iloc[: , 1:]
df_3 = strings_only.iloc[: , 5:]
df = pd.merge(df_2, df_3, left_index=True, right_index=True)

<ipython-input-3-618fc6e49d9a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strings_only[col+' Num'] = [hash(val) for val in strings_only[col]]


In [4]:
df['Source Port'] = df['Source Port'].fillna(0)
df['Destination Port'] = df['Destination Port'].fillna(0)
df = df.dropna()

#len(df['No.'])
for col in df:
    print(col, len(df[col]))

No. 1921093
MAC Source 1921093
Length 1921093
Source Port 1921093
Destination Port 1921093
Delta Time 1921093
MAC Destination Num 1921093
Protocol Num 1921093
Info Num 1921093


In [6]:
per_of_rows = 0.25
rows = int(len(df)*per_of_rows)
df = df[:rows]

In [7]:
X = df[['Length', 'Delta Time', 'Source Port', 'Destination Port', 'Info Num', 'Protocol Num', 'MAC Destination Num']]
#dataset[['Source','MAC Destination','Destination','Length','Source Port','Destination Port','Protocol', 'Delta Time']]
y = df['MAC Source']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# #Create a Gaussian Classifier
# clf=RandomForestClassifier(n_estimators=100)

# #Train the model using the training sets y_pred=clf.predict(X_test)
# clf.fit(X_train,y_train)

# y_pred=clf.predict(X_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [10]:
# Import required libraries for performance metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

knn = KNeighborsClassifier()
dtr = DecisionTreeClassifier(random_state=24)
rfc = RandomForestClassifier(random_state=24)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()

models = [knn, dtr, rfc, xboost, gnb]
models_scores_table = pd.DataFrame(index=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Time'])

for model in models:
    start = time.perf_counter()
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    scores = [
                accuracy_score(y_test, y_pred),
                precision_score(y_test, y_pred,average='weighted'),
                recall_score(y_test, y_pred,average='weighted'),
                f1_score(y_test, y_pred,average='weighted')
             ]
    delta = time.perf_counter() - start
    scores.append(delta)
    models_scores_table[type(model).__name__] = scores
    print(type(model).__name__, delta)

# Add 'Best Score' column
models_scores_table['Best Score'] = np.where(models_scores_table.index != 'Time', models_scores_table.idxmax(axis=1), models_scores_table.idxmin(axis=1))

C:\Users\carlo\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
KNeighborsClassifier 49.008616999999504
DecisionTreeClassifier 4.217784299999039
RandomForestClassifier 38.09459969999989
C:\Users\carlo\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[15:24:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_

In [14]:
# models_scores_table.to_csv('results.csv')
models_scores_table

,KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,XGBClassifier,GaussianNB,Best Score
Accuracy,0.826592,0.981635,0.983468,0.987264,0.554247,XGBClassifier
Precision,0.826824,0.981672,0.983353,0.987276,0.516756,XGBClassifier
Recall,0.826592,0.981635,0.983468,0.987264,0.554247,XGBClassifier
F1 Score,0.824066,0.981650,0.983293,0.987128,0.512725,XGBClassifier
Time,49.008617,4.217784,38.094600,239.456425,3.385906,GaussianNB


In [818]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

df = dataset[dataset['MAC Source']=='00:0c:29:9d:9e:9e']
df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
packets_per_sec = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
df_small_noise = packets_per_sec[:-1].to_frame().reset_index()
df_clean = df_small_noise.fillna(df_small_noise.mean())

<ipython-input-818-af6817c5455c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
<ipython-input-818-af6817c5455c>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_clean = df_small_noise.fillna(df_small_noise.mean())


In [819]:
model=IsolationForest(n_estimators=10, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(df_clean[['Delta Time']])

IsolationForest(contamination=0.1, n_estimators=10)

In [821]:
# # i = 11
# while True:
#     if -1 not in model.predict(df_clean[['Delta Time']]): break
#     # # model.set_params(n_estimators=i)
#     df_clean['scores']=model.decision_function(df_clean[['Delta Time']])
#     df_clean['anomaly']=model.predict(df_clean[['Delta Time']])
#     anomaly=df_clean.loc[df_clean['anomaly']==-1]
#     anomaly = anomaly.fillna(anomaly.mean())
#     model.fit(anomaly[['Delta Time']])
#     # print(len(anomaly))
#     #print('yeet')
# #anomaly.empty
# # model.fit(anomaly[['Delta Time']])

df_clean['scores']=model.decision_function(df_clean[['Delta Time']])
df_clean['anomaly']=model.predict(df_clean[['Delta Time']])
anomaly=df_clean.loc[df_clean['anomaly']==-1]
anomaly = anomaly.fillna(anomaly.mean())
# if len(anomaly) == 1:
#     anomaly = anomaly.append(anomaly)
model.fit(anomaly[['Delta Time']])
len(anomaly)

<ipython-input-821-ce7d1882f83f>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  anomaly = anomaly.fillna(anomaly.mean())


4

In [792]:
# df_clean.fillna(anomaly.mean())
# df_clean['scores']=model.decision_function(df_clean[['Delta Time']])
#df_clean['anomaly']=model.predict(df_clean[['Delta Time']])
#anomaly=df_clean.loc[df_clean['anomaly']==-1]
#anomaly

In [836]:
df_mitm = pd.read_csv ('test2.csv')
df_mitm = df_mitm[df_mitm['MAC Source']=='00:0c:29:9d:9e:9e']
df_mitm['Epoch Time'] = pd.to_datetime(df_mitm['Epoch Time'], errors='coerce')
df_mitm = df_mitm.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
df_mitm = df_mitm[:-1].to_frame().reset_index().fillna(df_mitm.mean())

df_mitm['scores']=model.decision_function(df_mitm[['Delta Time']])
df_mitm['anomaly']=model.predict(df_mitm[['Delta Time']])
anomaly=df_mitm.loc[df_mitm['anomaly']==-1]
anomaly

# # threshold
# threshold_score = anomaly['scores'].mean()

# #anomalies
# anomaly_final = anomaly[anomaly['scores'] < threshold_score]
# anomaly_final 

,Epoch Time,Delta Time,scores,anomaly
